Работа с векторными базами данных

1. Загрузка датасета

In [1]:
from datasets import load_dataset
from sentence_transformers import SentenceTransformer
import numpy as np
import faiss
from tqdm import tqdm
from collections import defaultdict
import json
import time
from typing import List, Dict, Tuple


/Users/elenasmetanina/Desktop/Sbertech/rag-project/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
# Загрузка датасета
dataset = load_dataset("ai-forever/ria-news-retrieval")
print("Структура датасета:")
print(dataset)
print("\nДоступные разделы:", list(dataset.keys()))

# Проверяем структуру каждого раздела
for split_name in dataset.keys():
    print(f"\n{split_name}:")
    print(f"  Количество строк: {len(dataset[split_name])}")
    print(f"  Колонки: {dataset[split_name].column_names}")
    if len(dataset[split_name]) > 0:
        print(f"  Пример первой строки: {dataset[split_name][0]}")

Структура датасета:
DatasetDict({
    test: Dataset({
        features: ['query-id', 'corpus-id', 'score'],
        num_rows: 10000
    })
})

Доступные разделы: ['test']

test:
  Количество строк: 10000
  Колонки: ['query-id', 'corpus-id', 'score']
  Пример первой строки: {'query-id': '0', 'corpus-id': '670487', 'score': 1}


2. Исследование структуры датасета и подготовка данных

In [7]:
# Загружаем отдельные разделы датасета
# Для retrieval датасетов queries и corpus могут быть отдельными конфигурациями
queries = None
corpus = None
test = None

# Способ 1: Пробуем загрузить queries через конфигурацию 'queries'
try:
    print("Попытка загрузки queries через конфигурацию 'queries'...")
    # Пробуем разные варианты split
    try:
        queries = load_dataset("ai-forever/ria-news-retrieval", name="queries", split="queries")
        print("✓ Queries загружены (split='queries')")
    except:
        # Если split не нужен, загружаем весь датасет queries
        queries_dataset = load_dataset("ai-forever/ria-news-retrieval", name="queries")
        # Берем первый доступный split
        if queries_dataset:
            queries = list(queries_dataset.values())[0]
            print(f"✓ Queries загружены из конфигурации (split: {list(queries_dataset.keys())[0]})")
except Exception as e:
    print(f"Не удалось загрузить queries через конфигурацию: {e}")

# Способ 2: Пробуем загрузить corpus через разные конфигурации
# Функция для проверки, что corpus содержит тексты
def is_valid_corpus(corpus_data):
    """Проверяет, что corpus содержит текстовую колонку"""
    if corpus_data is None:
        return False
    text_columns = ['text', 'content', 'document', 'passage', 'body', 'article']
    return any(col in corpus_data.column_names for col in text_columns)

if corpus is None:
    # Пробуем разные варианты названий конфигураций для corpus
    config_names_to_try = ['corpus', 'documents', 'passages', 'docs', 'collection']
    
    for config_name in config_names_to_try:
        try:
            print(f"\nПопытка загрузки corpus через конфигурацию '{config_name}'...")
            corpus_dataset = load_dataset("ai-forever/ria-news-retrieval", name=config_name)
            if corpus_dataset:
                corpus_candidate = list(corpus_dataset.values())[0]
                if is_valid_corpus(corpus_candidate):
                    corpus = corpus_candidate
                    print(f"✓ Corpus загружен из конфигурации '{config_name}' (split: {list(corpus_dataset.keys())[0]})")
                    break
                else:
                    print(f"  Конфигурация '{config_name}' не содержит текстов (колонки: {corpus_candidate.column_names})")
        except Exception as e:
            print(f"  Не удалось загрузить через '{config_name}': {e}")
    
    # Если не нашли через другие конфигурации, пробуем 'default' (но проверяем, что это не test)
    if corpus is None:
        try:
            print("\nПопытка загрузки corpus через конфигурацию 'default'...")
            corpus_dataset = load_dataset("ai-forever/ria-news-retrieval", name="default")
            if corpus_dataset:
                corpus_candidate = list(corpus_dataset.values())[0]
                if is_valid_corpus(corpus_candidate):
                    corpus = corpus_candidate
                    print(f"✓ Corpus загружен из конфигурации 'default' (split: {list(corpus_dataset.keys())[0]})")
                else:
                    print(f"  Конфигурация 'default' не содержит текстов (колонки: {corpus_candidate.column_names})")
                    print(f"  Это похоже на файл соответствий, а не на corpus с текстами")
        except Exception as e:
            print(f"Не удалось загрузить corpus через 'default': {e}")

# Способ 3: Пробуем загрузить как отдельные splits из основного датасета
if queries is None or corpus is None:
    try:
        print("\nПопытка загрузки queries и corpus как отдельных splits...")
        full_dataset = load_dataset("ai-forever/ria-news-retrieval")
        print(f"Доступные разделы: {list(full_dataset.keys())}")
        
        # Ищем queries и corpus в доступных разделах
        for key in full_dataset.keys():
            key_lower = key.lower()
            if ('query' in key_lower or 'queries' in key_lower) and queries is None:
                queries = full_dataset[key]
                print(f"✓ Найден queries в разделе '{key}'")
            elif 'corpus' in key_lower and corpus is None:
                corpus = full_dataset[key]
                print(f"✓ Найден corpus в разделе '{key}'")
            elif ('test' in key_lower or 'default' in key_lower) and test is None:
                test = full_dataset[key]
                print(f"✓ Найден test/default в разделе '{key}'")
    except Exception as e:
        print(f"Ошибка при загрузке: {e}")

# Если test не загружен, используем из dataset
if test is None:
    test = dataset.get("test", None)

# Выводим информацию о загруженных данных
print("\n" + "="*80)
print("РЕЗУЛЬТАТЫ ЗАГРУЗКИ")
print("="*80)

if queries is not None:
    print(f"\n✓ Queries: {len(queries)} запросов")
    print(f"  Колонки: {queries.column_names}")
    if len(queries) > 0:
        print(f"  Пример: {queries[0]}")
else:
    print("\n⚠ Queries не найдены - возможно, нужно извлечь из test")

if corpus is not None:
    print(f"\n✓ Corpus: {len(corpus)} документов")
    print(f"  Колонки: {corpus.column_names}")
    if len(corpus) > 0:
        # Показываем пример, но проверяем наличие текстовой колонки
        text_cols = [c for c in corpus.column_names if c in ['text', 'content', 'document', 'passage', 'body', 'article']]
        if text_cols:
            sample_text = corpus[0][text_cols[0]]
            print(f"  Пример текста (первые 200 символов): {str(sample_text)[:200]}...")
        else:
            sample = str(corpus[0])
            print(f"  Пример записи: {sample[:200]}...")
            print(f"  ⚠ ВНИМАНИЕ: Не найдена текстовая колонка! Возможно, загружен не тот corpus.")
else:
    print("\n⚠ Corpus не найден!")
    print("  Возможные причины:")
    print("  1. Corpus может быть в другой конфигурации")
    print("  2. Нужно проверить документацию датасета на Hugging Face")
    print("  3. Возможно, corpus нужно загрузить отдельно")

if test is not None:
    print(f"\n✓ Test/Default: {len(test)} соответствий")
    print(f"  Колонки: {test.column_names}")
    if len(test) > 0:
        print(f"  Пример: {test[0]}")
else:
    print("\n⚠ Test/Default не найден")


Попытка загрузки queries через конфигурацию 'queries'...
✓ Queries загружены (split='queries')

Попытка загрузки corpus через конфигурацию 'corpus'...


Generating corpus split: 100%|██████████| 704344/704344 [00:02<00:00, 269339.04 examples/s]

✓ Corpus загружен из конфигурации 'corpus' (split: corpus)

РЕЗУЛЬТАТЫ ЗАГРУЗКИ

✓ Queries: 10000 запросов
  Колонки: ['_id', 'text']
  Пример: {'_id': '0', 'text': 'сми: планируется создание в рф несетевых магазинов возле жилых домов'}

✓ Corpus: 704344 документов
  Колонки: ['_id', 'title', 'text']
  Пример текста (первые 200 символов): премьер-министр украины, кандидат в президенты юлия тимошенко в воскресенье в прямом эфире украинского телеканала 1+1 заявила, что в случае ее победы на выборах президента юрий луценко будет работать...

✓ Test/Default: 10000 соответствий
  Колонки: ['query-id', 'corpus-id', 'score']
  Пример: {'query-id': '0', 'corpus-id': '670487', 'score': 1}


In [8]:
# Дополнительная проверка: смотрим все доступные конфигурации датасета
if corpus is None or not is_valid_corpus(corpus):
    print("\n" + "="*80)
    print("ПОИСК ПРАВИЛЬНОЙ КОНФИГУРАЦИИ ДЛЯ CORPUS")
    print("="*80)
    
    try:
        # Пробуем получить информацию о всех конфигурациях
        from huggingface_hub import dataset_info
        info = dataset_info("ai-forever/ria-news-retrieval")
        print(f"\nДоступные конфигурации в датасете:")
        if hasattr(info, 'configs'):
            for config_name in info.configs.keys():
                print(f"  - {config_name}")
        else:
            print("  (информация о конфигурациях недоступна)")
    except Exception as e:
        print(f"Не удалось получить информацию о конфигурациях: {e}")
    
    # Пробуем загрузить все возможные конфигурации и проверить их
    print("\nПроверяем все возможные конфигурации...")
    possible_configs = ['corpus', 'documents', 'passages', 'docs', 'collection', 'default', 'queries', 'test']
    
    for config_name in possible_configs:
        try:
            config_dataset = load_dataset("ai-forever/ria-news-retrieval", name=config_name)
            if config_dataset:
                for split_name, split_data in config_dataset.items():
                    print(f"\n  Конфигурация '{config_name}', split '{split_name}':")
                    print(f"    Колонки: {split_data.column_names}")
                    print(f"    Количество строк: {len(split_data)}")
                    if is_valid_corpus(split_data) and corpus is None:
                        corpus = split_data
                        print(f"    ✓ Это corpus с текстами! Используем его.")
                        break
        except:
            pass
    
    if corpus is None or not is_valid_corpus(corpus):
        print("\n⚠ Corpus с текстами не найден автоматически.")
        print("Рекомендации:")
        print("1. Проверьте документацию датасета: https://huggingface.co/datasets/ai-forever/ria-news-retrieval")
        print("2. Возможно, corpus нужно загрузить отдельным запросом")
        print("3. Или corpus может быть в другом формате/файле")


3. Выбор модели эмбеддингов

Модель преобразует текст в числовые векторы (эмбеддинги), чтобы можно было сравнивать вопросы с текстами корпуса по смыслу.

In [5]:
# Выбор модели эмбеддингов
# Можно использовать разные модели для сравнения результатов
MODEL_NAME = "sentence-transformers/distiluse-base-multilingual-cased-v1"
# Альтернативные модели:
# MODEL_NAME = "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2"
# MODEL_NAME = "sentence-transformers/LaBSE"

print(f"Загрузка модели: {MODEL_NAME}")
embedding_model = SentenceTransformer(MODEL_NAME)
print(f"✓ Модель загружена. Размерность эмбеддингов: {embedding_model.get_sentence_embedding_dimension()}")

Загрузка модели: sentence-transformers/distiluse-base-multilingual-cased-v1
✓ Модель загружена. Размерность эмбеддингов: 512


4. Создание векторной базы данных (FAISS)

Наполняем векторную базу эмбеддингами текстов из корпуса.

In [9]:
# Подготовка данных корпуса
# Определяем, какая колонка содержит текст
if corpus is None:
    raise ValueError("Corpus не загружен! Проверьте структуру датасета.")

# Проверяем, что corpus содержит текстовую колонку
print(f"Колонки в corpus: {corpus.column_names}")

# Пробуем найти колонку с текстом
text_column = None
text_column_candidates = ['text', 'content', 'document', 'passage', 'body', 'article', 'sentence']
for col in text_column_candidates:
    if col in corpus.column_names:
        text_column = col
        break

if text_column is None:
    # Если не нашли стандартную колонку, ищем колонки, которые не являются ID или числовыми
    non_id_cols = [c for c in corpus.column_names 
                   if 'id' not in c.lower() 
                   and c not in ['score', 'label', 'index']]
    
    if non_id_cols:
        # Берем самую длинную колонку (вероятно, это текст)
        text_column = non_id_cols[0]
        print(f"⚠ ВНИМАНИЕ: Не найдена стандартная текстовая колонка!")
        print(f"⚠ Используем колонку '{text_column}' как текст")
        print(f"⚠ Убедитесь, что это правильная колонка с текстами документов!")
    else:
        raise ValueError(
            f"Не найдена текстовая колонка в corpus! "
            f"Доступные колонки: {corpus.column_names}. "
            f"Возможно, загружен не тот corpus (например, файл соответствий вместо текстов)."
        )

print(f"Используем колонку '{text_column}' для текстов корпуса")

# Извлекаем тексты и создаем маппинг ID -> текст
corpus_texts = []
corpus_id_mapping = {}  # индекс в векторах -> corpus_id

# Определяем колонку с ID корпуса
corpus_id_column = None
for col in ['id', 'corpus-id', 'corpus_id', '_id']:
    if col in corpus.column_names:
        corpus_id_column = col
        break

print(f"Колонка ID корпуса: {corpus_id_column if corpus_id_column else 'индекс'}")

# Создаем список текстов и маппинг
for idx, item in enumerate(corpus):
    text = item[text_column]
    corpus_texts.append(text)
    corpus_id = item[corpus_id_column] if corpus_id_column else idx
    corpus_id_mapping[idx] = corpus_id

print(f"Загружено {len(corpus_texts)} документов из корпуса")

# Создаем эмбеддинги для корпуса
print("\nСоздание эмбеддингов для корпуса...")
start_time = time.time()
corpus_embeddings = embedding_model.encode(
    corpus_texts,
    show_progress_bar=True,
    batch_size=32,
    convert_to_numpy=True
)
print(f"✓ Эмбеддинги созданы за {time.time() - start_time:.2f} секунд")
print(f"Размерность: {corpus_embeddings.shape}")

# Нормализуем векторы для косинусного сходства
corpus_embeddings = corpus_embeddings / np.linalg.norm(corpus_embeddings, axis=1, keepdims=True)

# Создаем FAISS индекс
dimension = corpus_embeddings.shape[1]
print(f"\nСоздание FAISS индекса (размерность: {dimension})...")
index = faiss.IndexFlatIP(dimension)  # Inner Product для нормализованных векторов = косинусное сходство

# Добавляем векторы в индекс
index.add(corpus_embeddings.astype('float32'))
print(f"✓ В векторную базу добавлено {index.ntotal} векторов")


Колонки в corpus: ['_id', 'title', 'text']
Используем колонку 'text' для текстов корпуса
Колонка ID корпуса: _id
Загружено 704344 документов из корпуса

Создание эмбеддингов для корпуса...


Batches: 100%|██████████| 22011/22011 [48:44<00:00,  7.53it/s] 


✓ Эмбеддинги созданы за 2945.89 секунд
Размерность: (704344, 512)

Создание FAISS индекса (размерность: 512)...
✓ В векторную базу добавлено 704344 векторов


In [10]:
# Подготовка queries и test данных
if queries is None:
    raise ValueError("Queries не загружены!")

if test is None:
    raise ValueError("Test/Default не загружен!")

# Определяем колонки
query_text_column = None
for col in ['text', 'query', 'content']:
    if col in queries.column_names:
        query_text_column = col
        break
if query_text_column is None:
    query_text_column = [c for c in queries.column_names if 'id' not in c.lower()][0]

query_id_column = None
for col in ['id', 'query-id', 'query_id', '_id']:
    if col in queries.column_names:
        query_id_column = col
        break

print(f"Колонка текста запросов: {query_text_column}")
print(f"Колонка ID запросов: {query_id_column if query_id_column else 'индекс'}")

# Создаем словарь query_id -> текст
query_dict = {}
for item in queries:
    q_id = item[query_id_column] if query_id_column else queries.index(item)
    q_text = item[query_text_column]
    query_dict[q_id] = q_text

print(f"Загружено {len(query_dict)} запросов")

# Создаем словарь query_id -> правильный corpus_id из test
# Определяем колонки в test
test_query_id_col = None
test_corpus_id_col = None

for col in ['query-id', 'query_id', 'query-id', 'query']:
    if col in test.column_names:
        test_query_id_col = col
        break

for col in ['corpus-id', 'corpus_id', 'corpus-id', 'corpus', 'document-id', 'document_id']:
    if col in test.column_names:
        test_corpus_id_col = col
        break

print(f"\nКолонка query_id в test: {test_query_id_col}")
print(f"Колонка corpus_id в test: {test_corpus_id_col}")

# Создаем маппинг query_id -> правильный corpus_id
ground_truth = {}
for item in test:
    q_id = item[test_query_id_col] if test_query_id_col else test.index(item)
    c_id = item[test_corpus_id_col] if test_corpus_id_col else None
    if c_id is not None:
        ground_truth[q_id] = c_id

print(f"Загружено {len(ground_truth)} соответствий query -> corpus")

# Создаем обратный маппинг corpus_id -> индекс в векторах
corpus_id_to_index = {corpus_id: idx for idx, corpus_id in corpus_id_mapping.items()}

print(f"\n✓ Данные подготовлены для оценки")


Колонка текста запросов: text
Колонка ID запросов: _id
Загружено 10000 запросов

Колонка query_id в test: query-id
Колонка corpus_id в test: corpus-id
Загружено 10000 соответствий query -> corpus

✓ Данные подготовлены для оценки


In [11]:
def evaluate_retrieval(
    query_dict: Dict,
    ground_truth: Dict,
    corpus_id_to_index: Dict,
    corpus_id_mapping: Dict,
    corpus_texts: List[str],
    embedding_model: SentenceTransformer,
    index: faiss.Index,
    k_values: List[int] = [1, 3, 5, 10],
    verbose: bool = False,
    log_errors: bool = True
) -> Dict:
    """
    Оценка качества поиска в векторной базе
    
    Args:
        query_dict: словарь query_id -> текст запроса
        ground_truth: словарь query_id -> правильный corpus_id
        corpus_id_to_index: маппинг corpus_id -> индекс в векторах
        corpus_id_mapping: маппинг индекс -> corpus_id
        corpus_texts: список текстов корпуса
        embedding_model: модель для создания эмбеддингов
        index: FAISS индекс
        k_values: список значений k для top-k оценки
        verbose: подробное логирование
        log_errors: логировать ошибки
    
    Returns:
        словарь с метриками оценки
    """
    # Статистика
    stats = {
        'total_queries': 0,
        'processed_queries': 0,
        'skipped_queries': 0,
        'correct_by_k': {k: 0 for k in k_values},
        'errors': []
    }
    
    # Создаем эмбеддинги для всех запросов
    query_ids = list(query_dict.keys())
    query_texts = [query_dict[q_id] for q_id in query_ids]
    
    print(f"\nСоздание эмбеддингов для {len(query_texts)} запросов...")
    query_embeddings = embedding_model.encode(
        query_texts,
        show_progress_bar=True,
        batch_size=32,
        convert_to_numpy=True
    )
    query_embeddings = query_embeddings / np.linalg.norm(query_embeddings, axis=1, keepdims=True)
    print("✓ Эмбеддинги запросов созданы")
    
    # Прогон по всем запросам
    print(f"\nВыполнение поиска для {len(query_ids)} запросов...")
    max_k = max(k_values)
    
    for i, (query_id, query_text, query_emb) in enumerate(tqdm(
        zip(query_ids, query_texts, query_embeddings),
        total=len(query_ids),
        desc="Обработка запросов"
    )):
        stats['total_queries'] += 1
        
        # Проверяем, есть ли правильный ответ для этого запроса
        if query_id not in ground_truth:
            stats['skipped_queries'] += 1
            if verbose:
                print(f"⚠ Query {query_id}: нет правильного ответа в ground_truth, пропускаем")
            continue
        
        correct_corpus_id = ground_truth[query_id]
        
        # Проверяем, есть ли правильный corpus_id в нашей базе
        if correct_corpus_id not in corpus_id_to_index:
            stats['skipped_queries'] += 1
            if verbose:
                print(f"⚠ Query {query_id}: правильный corpus_id {correct_corpus_id} не найден в базе")
            continue
        
        stats['processed_queries'] += 1
        correct_index = corpus_id_to_index[correct_corpus_id]
        
        # Поиск в векторной базе
        try:
            query_emb_reshaped = query_emb.reshape(1, -1).astype('float32')
            distances, indices = index.search(query_emb_reshaped, max_k)
            
            # Получаем corpus_id для найденных индексов
            found_indices = indices[0]
            found_corpus_ids = [corpus_id_mapping[idx] for idx in found_indices]
            
            # Проверяем для каждого k
            for k in k_values:
                top_k_ids = found_corpus_ids[:k]
                if correct_corpus_id in top_k_ids:
                    stats['correct_by_k'][k] += 1
            
            # Логируем ошибки, если правильный ответ не в top-3
            if log_errors and correct_corpus_id not in found_corpus_ids[:3]:
                error_info = {
                    'query_id': query_id,
                    'query_text': query_text[:200] + ('...' if len(query_text) > 200 else ''),
                    'correct_corpus_id': correct_corpus_id,
                    'top_3_corpus_ids': found_corpus_ids[:3],
                    'top_3_snippets': [corpus_texts[found_indices[j]][:150] + '...' 
                                      for j in range(min(3, len(found_indices)))]
                }
                stats['errors'].append(error_info)
            
            # Подробное логирование
            if verbose and i < 5:  # Показываем первые 5 запросов
                print(f"\n--- Query {i+1} (ID: {query_id}) ---")
                print(f"Текст: {query_text[:150]}{'...' if len(query_text) > 150 else ''}")
                print(f"Правильный corpus_id: {correct_corpus_id}")
                print(f"Top-{max_k} результаты:")
                for rank, (idx, c_id, dist) in enumerate(zip(found_indices, found_corpus_ids, distances[0]), 1):
                    is_correct = "✓" if c_id == correct_corpus_id else "✗"
                    snippet = corpus_texts[idx][:100] + ('...' if len(corpus_texts[idx]) > 100 else '')
                    print(f"  {rank}. {is_correct} ID {c_id} (расстояние: {dist:.4f}): {snippet}")
        
        except Exception as e:
            print(f"❌ Ошибка при обработке query {query_id}: {e}")
            stats['errors'].append({
                'query_id': query_id,
                'error': str(e)
            })
    
    # Вычисляем метрики
    if stats['processed_queries'] > 0:
        stats['accuracy_by_k'] = {
            k: stats['correct_by_k'][k] / stats['processed_queries'] 
            for k in k_values
        }
    else:
        stats['accuracy_by_k'] = {k: 0.0 for k in k_values}
    
    return stats

# Запускаем оценку
print("=" * 80)
print("НАЧАЛО ОЦЕНКИ КАЧЕСТВА ПОИСКА")
print("=" * 80)
print(f"Модель: {MODEL_NAME}")
print(f"Векторная база: FAISS (IndexFlatIP)")
print(f"Количество документов в корпусе: {len(corpus_texts)}")
print(f"Количество запросов: {len(query_dict)}")
print(f"Количество соответствий в ground_truth: {len(ground_truth)}")

evaluation_stats = evaluate_retrieval(
    query_dict=query_dict,
    ground_truth=ground_truth,
    corpus_id_to_index=corpus_id_to_index,
    corpus_id_mapping=corpus_id_mapping,
    corpus_texts=corpus_texts,
    embedding_model=embedding_model,
    index=index,
    k_values=[1, 3, 5, 10],
    verbose=True,  # Показываем первые 5 запросов подробно
    log_errors=True
)


НАЧАЛО ОЦЕНКИ КАЧЕСТВА ПОИСКА
Модель: sentence-transformers/distiluse-base-multilingual-cased-v1
Векторная база: FAISS (IndexFlatIP)
Количество документов в корпусе: 704344
Количество запросов: 10000
Количество соответствий в ground_truth: 10000

Создание эмбеддингов для 10000 запросов...


Batches: 100%|██████████| 313/313 [00:11<00:00, 27.34it/s]


✓ Эмбеддинги запросов созданы

Выполнение поиска для 10000 запросов...


Обработка запросов:   0%|          | 6/10000 [00:00<08:30, 19.58it/s]


--- Query 1 (ID: 0) ---
Текст: сми: планируется создание в рф несетевых магазинов возле жилых домов
Правильный corpus_id: 670487
Top-10 результаты:
  1. ✗ ID 240745 (расстояние: 0.4320): требования к строительству жилья на дачных участках в московской области планируется ужесточить пос...
  2. ✗ ID 240707 (расстояние: 0.4320): требования к строительству жилья на дачных участках в московской области планируется ужесточить пос...
  3. ✗ ID 593361 (расстояние: 0.4300): почти 40 тысяч квадратных метров жилья планируется построить на месте шести пятиэтажек в районе бес...
  4. ✗ ID 543337 (расстояние: 0.4248): жители многоэтажки на красном проспекте в новосибирске выступили против проведения строительных рабо...
  5. ✗ ID 422846 (расстояние: 0.4233): «торговый дом перекресток» приступил к отделочным работам в зоне супермаркета торгового центра «изм...
  6. ✗ ID 465507 (расстояние: 0.4229): каждый пятый британский магазин закроется к 2018 году в связи с ростом популярности интернет-т

Обработка запросов: 100%|██████████| 10000/10000 [03:50<00:00, 43.30it/s]


7. Вывод результатов оценки


In [12]:
# Вывод итоговой статистики
print("\n" + "=" * 80)
print("ИТОГОВАЯ СТАТИСТИКА ОЦЕНКИ")
print("=" * 80)
print(f"\nМодель эмбеддингов: {MODEL_NAME}")
print(f"Векторная база: FAISS (IndexFlatIP)")
print(f"\nОбщая статистика:")
print(f"  Всего запросов: {evaluation_stats['total_queries']}")
print(f"  Обработано запросов: {evaluation_stats['processed_queries']}")
print(f"  Пропущено запросов: {evaluation_stats['skipped_queries']}")

print(f"\nТочность (Accuracy) по Top-K:")
for k in sorted(evaluation_stats['accuracy_by_k'].keys()):
    correct = evaluation_stats['correct_by_k'][k]
    total = evaluation_stats['processed_queries']
    accuracy = evaluation_stats['accuracy_by_k'][k]
    print(f"  Top-{k:2d}: {correct:5d}/{total:5d} = {accuracy*100:6.2f}%")

print(f"\nОшибки (запросы, где правильный ответ не в Top-3): {len(evaluation_stats['errors'])}")

# Показываем примеры ошибок
if evaluation_stats['errors']:
    print(f"\nПримеры ошибок (первые 5):")
    for i, error in enumerate(evaluation_stats['errors'][:5], 1):
        print(f"\n  Ошибка {i}:")
        print(f"    Query ID: {error.get('query_id', 'N/A')}")
        print(f"    Текст запроса: {error.get('query_text', 'N/A')}")
        print(f"    Правильный corpus_id: {error.get('correct_corpus_id', 'N/A')}")
        print(f"    Top-3 найденные corpus_id: {error.get('top_3_corpus_ids', [])}")
        if 'top_3_snippets' in error:
            print(f"    Snippets найденных результатов:")
            for j, snippet in enumerate(error['top_3_snippets'], 1):
                print(f"      {j}. {snippet}")

# Сохраняем результаты в JSON
results_summary = {
    'model': MODEL_NAME,
    'vector_store': 'FAISS (IndexFlatIP)',
    'corpus_size': len(corpus_texts),
    'total_queries': evaluation_stats['total_queries'],
    'processed_queries': evaluation_stats['processed_queries'],
    'accuracy': evaluation_stats['accuracy_by_k'],
    'num_errors': len(evaluation_stats['errors'])
}

print(f"\n✓ Результаты сохранены в переменную 'results_summary'")
print(f"  Для сохранения в файл используйте: json.dump(results_summary, open('results.json', 'w'), indent=2)")



ИТОГОВАЯ СТАТИСТИКА ОЦЕНКИ

Модель эмбеддингов: sentence-transformers/distiluse-base-multilingual-cased-v1
Векторная база: FAISS (IndexFlatIP)

Общая статистика:
  Всего запросов: 10000
  Обработано запросов: 10000
  Пропущено запросов: 0

Точность (Accuracy) по Top-K:
  Top- 1:  3141/10000 =  31.41%
  Top- 3:  4515/10000 =  45.15%
  Top- 5:  5091/10000 =  50.91%
  Top-10:  5811/10000 =  58.11%

Ошибки (запросы, где правильный ответ не в Top-3): 5485

Примеры ошибок (первые 5):

  Ошибка 1:
    Query ID: 0
    Текст запроса: сми: планируется создание в рф несетевых магазинов возле жилых домов
    Правильный corpus_id: 670487
    Top-3 найденные corpus_id: ['240745', '240707', '593361']
    Snippets найденных результатов:
      1. требования к строительству жилья на дачных участках в московской области планируется ужесточить после внесения ряда поправок в федеральное законодате...
      2. требования к строительству жилья на дачных участках в московской области планируется ужесточить

In [14]:
json.dump(results_summary, open('results.json', 'w'), indent=2)


8. Эксперименты с разными моделями и параметрами

Можно попробовать разные модели эмбеддингов и сравнить результаты.


In [18]:
# Функция для быстрого сравнения разных моделей
def compare_models(
    model_names: List[str],
    query_dict: Dict,
    ground_truth: Dict,
    corpus_texts: List[str],
    corpus_id_mapping: Dict,
    corpus_id_to_index: Dict,
    k_values: List[int] = [1, 3]
):
    """
    Сравнение разных моделей эмбеддингов
    """
    results_comparison = {}
    
    for model_name in model_names:
        print(f"\n{'='*80}")
        print(f"Тестирование модели: {model_name}")
        print(f"{'='*80}")
        
        # Загружаем модель
        print(f"Загрузка модели...")
        model = SentenceTransformer(model_name)
        
        # Создаем эмбеддинги корпуса
        print(f"Создание эмбеддингов корпуса...")
        corpus_emb = model.encode(corpus_texts, show_progress_bar=True, batch_size=32, convert_to_numpy=True)
        corpus_emb = corpus_emb / np.linalg.norm(corpus_emb, axis=1, keepdims=True)
        
        # Создаем индекс
        dimension = corpus_emb.shape[1]
        index = faiss.IndexFlatIP(dimension)
        index.add(corpus_emb.astype('float32'))
        
        # Оцениваем
        stats = evaluate_retrieval(
            query_dict=query_dict,
            ground_truth=ground_truth,
            corpus_id_to_index=corpus_id_to_index,
            corpus_id_mapping=corpus_id_mapping,
            corpus_texts=corpus_texts,
            embedding_model=model,
            index=index,
            k_values=k_values,
            verbose=False,
            log_errors=False
        )
        
        results_comparison[model_name] = {
            'accuracy': stats['accuracy_by_k'],
            'processed': stats['processed_queries']
        }
        
        print(f"\nРезультаты для {model_name}:")
        for k in k_values:
            print(f"  Top-{k}: {stats['accuracy_by_k'][k]*100:.2f}%")
    
    # Сравнительная таблица
    print(f"\n{'='*80}")
    print("СРАВНИТЕЛЬНАЯ ТАБЛИЦА")
    print(f"{'='*80}")
    print(f"{'Модель':<60} {'Top-1':<10} {'Top-3':<10}")
    print("-" * 80)
    for model_name, results in results_comparison.items():
        top1 = results['accuracy'].get(1, 0) * 100
        top3 = results['accuracy'].get(3, 0) * 100
        print(f"{model_name:<60} {top1:>6.2f}%   {top3:>6.2f}%")
    
    return results_comparison

# Раскомментируйте для сравнения разных моделей:
alternative_models = [
    "sentence-transformers/distiluse-base-multilingual-cased-v1",
     "sentence-transformers/paraphrase-multilingual-MiniLM-L12-v2",
     "sentence-transformers/LaBSE"
 ]
comparison_results = compare_models(
     model_names=alternative_models,
     query_dict=query_dict,
     ground_truth=ground_truth,
     corpus_texts=corpus_texts,
     corpus_id_mapping=corpus_id_mapping,
     corpus_id_to_index=corpus_id_to_index,
     k_values=[1, 3, 5]
 )
# Если вы уже ранее запускали сравнение моделей и получили переменную comparison_results,
# то вот код для вывода сравнительной таблицы отдельно.




Тестирование модели: sentence-transformers/distiluse-base-multilingual-cased-v1
Загрузка модели...
Создание эмбеддингов корпуса...


Batches:  12%|█▏        | 2618/22011 [05:58<44:17,  7.30it/s]


KeyboardInterrupt: 